In [5]:
%pip install numpy
%pip install pandas

import numpy as np
import pandas as pd

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 11.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 6.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


## Preparação da base de dados da UCI: unificação dos arquivos de português e matemática

Opção A – Interseção (alunos que aparecem em ambos)

In [7]:
# Carregando os datasets
df_mat = pd.read_csv("/home/emanoel/mestrado/codificando/datasets/cortez/student-mat.csv", sep=';')
df_por = pd.read_csv("/home/emanoel/mestrado/codificando/datasets/cortez/student-por.csv", sep=';')

# Colunas usadas como chave
join_cols = ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus',
             'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian',
             'traveltime', 'studytime', 'failures', 'schoolsup',
             'famsup', 'paid', 'activities', 'nursery', 'higher',
             'internet', 'romantic']

# Fazendo o merge dos alunos em comum
df_merged = pd.merge(df_mat, df_por, on=join_cols, suffixes=('_mat', '_por'))


Opção B – União completa (todos os alunos, mesmo que estejam só em uma disciplina)

In [8]:
df_mat['subject'] = 'Math'
df_por['subject'] = 'Portuguese'

df_all = pd.concat([df_mat, df_por], ignore_index=True)

Criação do student_id com base nas chaves compostas:

In [9]:
df_merged["student_id"] = df_merged[join_cols].astype(str).agg('_'.join, axis=1)

Padronização UCI (D1)

In [10]:
# Adicionar coluna indicativa de curso
df_por["course"] = "Portuguese"
df_mat["course"] = "Math"

# Unir os datasets em um único DataFrame (D1)
df_d1 = pd.concat([df_por, df_mat], ignore_index=True)

# Selecionar colunas relevantes
df_d1_reduced = df_d1[["sex", "age", "failures", "G1", "G2", "G3"]].copy()

# Renomear colunas para padronizar com D3
df_d1_reduced.rename(columns={
    "sex": "gender",
    "age": "age",
    "failures": "num_of_prev_attempts",
    "G1": "G1_score",
    "G2": "G2_score",
    "G3": "score"
}, inplace=True)

# Criar faixa etária (age_band) como no OULAD
df_d1_reduced["age_band"] = pd.cut(
    df_d1_reduced["age"],
    bins=[0, 35, 55, 100],
    labels=["0-35", "35-55", "55<"]
)

# Criar classificação final baseada na nota final (score)
df_d1_reduced["final_result"] = pd.cut(
    df_d1_reduced["score"],
    bins=[-1, 10, 13, 16, 20],
    labels=["Fail", "Pass", "Good", "Excellent"]
)

# Salvar o resultado
df_d1_reduced.to_csv("d1_padronizado.csv", index=False)

# Verificar as primeiras linhas
print(df_d1_reduced.head())


  gender  age  num_of_prev_attempts  G1_score  G2_score  score age_band  \
0      F   18                     0         0        11     11     0-35   
1      F   17                     0         9        11     11     0-35   
2      F   15                     0        12        13     12     0-35   
3      F   15                     0        14        14     14     0-35   
4      F   16                     0        11        13     13     0-35   

  final_result  
0         Pass  
1         Pass  
2         Pass  
3         Good  
4         Pass  


## Preparação da base de dados OULAD

In [11]:
# Carregar arquivos necessários
info = pd.read_csv("/home/emanoel/mestrado/codificando/datasets/OULAD/studentInfo.csv")
assessments = pd.read_csv("/home/emanoel/mestrado/codificando/datasets/OULAD/assessments.csv")
student_assessments = pd.read_csv("/home/emanoel/mestrado/codificando/datasets/OULAD/studentAssessment.csv")

# Calcular a média das notas por aluno
merged = student_assessments.merge(assessments, on="id_assessment")
student_scores = merged.groupby("id_student")["score"].mean().reset_index()

# Juntar com os dados demográficos
d2 = info.merge(student_scores, on="id_student", how="left")

# Harmonizar resultado final
d2["final_result"] = d2["final_result"].replace({
    "Withdrawn": "Fail",
    "Distinction": "Excellent"
})

# Selecionar e renomear colunas
d2_padronizado = d2[[
    "gender", "age_band", "num_of_prev_attempts", "score", "final_result"
]].copy()

# Salvar como CSV final
d2_padronizado.to_csv("d2_padronizado.csv", index=False)

# Visualizar amostra
print(d2_padronizado.head())


  gender age_band  num_of_prev_attempts  score final_result
0      M     55<=                     0   82.0         Pass
1      F    35-55                     0   66.4         Pass
2      F    35-55                     0    NaN         Fail
3      F    35-55                     0   76.0         Pass
4      F     0-35                     0   54.4         Pass


In [18]:
# Carregar o dataset sintético 
df_d3 = pd.read_csv("/home/emanoel/mestrado/codificando/datasets/dataset_sintetico.csv")

# Copiar o dataframe original
df_d3_enriched = df_d3.copy()

# Gerar colunas sintéticas
np.random.seed(42)
n = len(df_d3_enriched)

# Adicionar gender
df_d3_enriched["gender"] = np.random.choice(["Male", "Female"], size=n)

# Adicionar age_band
df_d3_enriched["age_band"] = np.random.choice(["0-35", "35-55", "55<"], size=n)

# Adicionar num_of_prev_attempts
df_d3_enriched["num_of_prev_attempts"] = np.random.randint(0, 4, size=n)

# Calcular score como média das notas
nota_cols = [
    'Números Inteiros', 'Frações', 'Equações', 'Geometria Básica',
    'Funções', 'Trigonometria', 'Probabilidade', 'Estatística'
]
df_d3_enriched["score"] = df_d3_enriched[nota_cols].mean(axis=1)

# Gerar final_result com base no score
df_d3_enriched["final_result"] = pd.cut(
    df_d3_enriched["score"],
    bins=[-1, 10, 13, 16, 20],
    labels=["Fail", "Pass", "Good", "Excellent"]
)

# Selecionar colunas padronizadas
colunas_finais = ["gender", "age_band", "num_of_prev_attempts", "score", "final_result"]
df_d3_padronizado = df_d3_enriched[colunas_finais]

# Salvar CSV
output_path = "dataset_sintetico_padronizado.csv"
df_d3_padronizado.to_csv(output_path, index=False)

df_d3_padronizado.head()



,gender,age_band,num_of_prev_attempts,score,final_result
0,Male,55<,3,6.775,Fail
1,Female,35-55,0,5.725,Fail
2,Male,55<,1,5.850,Fail
3,Male,55<,2,6.550,Fail
4,Male,0-35,0,6.200,Fail


O dataset sintético (D3) foi enriquecido com os seguintes atributos compatíveis com D1 e D2:

| Novo atributo          | Fonte               | Descrição                                              |
| ---------------------- | ------------------- | ------------------------------------------------------ |
| `gender`               | Gerado              | Sexo (`M` ou `F`)                                      |
| `age_band`             | Gerado              | Faixa etária (`0-35`, `35-55`, `55<`)                  |
| `num_of_prev_attempts` | Gerado              | Número de tentativas anteriores                        |
| `score`                | Calculado           | Média das notas em tópicos de matemática               |
| `final_result`         | Derivado de `score` | Classificação como `Fail`, `Pass`, `Good`, `Excellent` |


Unificação dos três datasets

In [17]:
import pandas as pd

# Carregar os datasets padronizados
d1 = pd.read_csv("d1_padronizado.csv")
d2 = pd.read_csv("d2_padronizado.csv")           # Assumindo que você criou este
d3 = pd.read_csv("dataset_sintetico_padronizado.csv")  # Já preparado anteriormente

# Adicionar origem de cada dataset
d1["dataset"] = "D1"
d2["dataset"] = "D2"
d3["dataset"] = "D3"

# Manter somente colunas comuns
colunas_comuns = ["gender", "age_band", "num_of_prev_attempts", "score", "final_result", "dataset"]

# Unificar os três datasets
df_unificado = pd.concat([d1[colunas_comuns], d2[colunas_comuns], d3[colunas_comuns]], ignore_index=True)

# Salvar a base unificada
df_unificado.to_csv("dados_unificados.csv", index=False)

# Visualizar amostra
print(df_unificado.sample(5))


      gender age_band  num_of_prev_attempts      score final_result dataset
24652      F    35-55                     0  85.333333         Pass      D2
5183       F     0-35                     0  88.818182         Pass      D2
18283      M     0-35                     0        NaN         Fail      D2
26701      M     0-35                     0  86.000000         Fail      D2
25467      M     0-35                     1        NaN         Fail      D2


In [ ]:
df_unificado

,gender,age_band,num_of_prev_attempts,score,final_result,dataset
0,F,0-35,0,11.0000,Pass,D1
1,F,0-35,0,11.0000,Pass,D1
2,F,0-35,0,12.0000,Pass,D1
3,F,0-35,0,14.0000,Good,D1
4,F,0-35,0,13.0000,Pass,D1
...,...,...,...,...,...,...
33682,Female,35-55,0,5.5250,Fail,D3
33683,Female,55<,3,6.1625,Fail,D3
33684,Male,0-35,2,7.1000,Fail,D3
33685,Female,35-55,2,5.3000,Fail,D3
